In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from tensorflow.keras.models import Sequential, save_model, load_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.utils import get_custom_objects
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, f1_score
import joblib
import scipy.stats as st
import threading
import time

# Register custom metric with Keras
def f1_metric(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true * y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1 - y_true) * (1 - y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1 - y_true) * y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true * (1 - y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2 * p * r / (p + r + K.epsilon())
    return K.mean(f1)

get_custom_objects().update({"f1_metric": f1_metric})

# Global Variables
dropout_value = 0.5
epochs_value = 10000
patience_value = 100
random_state_value = 50
max_search_time = 300  # Maximum time for each RandomizedSearchCV in seconds (5 minutes)

# Custom timeout handler using threading
class TimeoutException(Exception):
    pass

def run_with_timeout(func, args=(), kwargs={}, timeout=300):
    result = [None]
    exception = [None]
    
    def wrapper():
        try:
            result[0] = func(*args, **kwargs)
        except Exception as e:
            exception[0] = e
    
    thread = threading.Thread(target=wrapper)
    thread.start()
    thread.join(timeout)
    
    if thread.is_alive():
        thread.join()
        raise TimeoutException("Timeout reached")
    
    if exception[0]:
        raise exception[0]
    
    return result[0]

# Load the training data
print("Loading training data...")
train_file_path = 'train.csv'  # replace with your actual file path
train_data = pd.read_csv(train_file_path)
print("Training data loaded.")

# Separate features and target variable
X_train = train_data.drop(columns=['Target'])
y_train = train_data['Target']

# Handle missing values
X_train = X_train.fillna(X_train.mean())

# Normalize the features
print("Normalizing features...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
print("Features normalized.")

# Build the neural network model
print("Building neural network model...")
def build_nn_model(input_dim):
    model = Sequential()
    model.add(Dense(128, input_dim=input_dim, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_value))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_value))
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_value))
    model.add(Dense(16, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=[f1_metric])
    return model

# Initialize the neural network model
nn_model = build_nn_model(X_train_scaled.shape[1])

# Train the neural network model
print("Training neural network model...")
early_stopping = EarlyStopping(monitor='val_loss', patience=patience_value, restore_best_weights=True, min_delta=0.001)
model_checkpoint = ModelCheckpoint('best_nn_model.h5', monitor='val_loss', save_best_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001, verbose=1)

history = nn_model.fit(X_train_scaled, y_train, validation_split=0.2, epochs=epochs_value, batch_size=32, callbacks=[early_stopping, model_checkpoint, reduce_lr])

# Load the best neural network model
print("Loading best neural network model...")
nn_model.load_weights('best_nn_model.h5')

# Create a wrapper for the neural network model to use it in the StackingClassifier
class NeuralNetworkClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, model):
        self.model = model
        self.classes_ = [0, 1]  # Add the classes_ attribute

    def fit(self, X, y, epochs=100, batch_size=32, verbose=0):
        self.model.fit(X, y, epochs=epochs, batch_size=batch_size, verbose=verbose)
        return self

    def predict(self, X):
        return (self.model.predict(X) > 0.5).astype(int).reshape(-1)

    def predict_proba(self, X):
        probas = self.model.predict(X)
        return np.hstack(((1 - probas), probas))

# Initialize the classifiers with hyperparameter tuning (excluding SVC and GBC)
print("Initializing and tuning classifiers...")
nn_wrapper = NeuralNetworkClassifier(nn_model)

param_dist = {
    'rf': {
        'n_estimators': st.randint(50, 200),
        'max_features': ['auto', 'sqrt', 'log2'],
        'max_depth': st.randint(4, 20),
        'criterion': ['gini', 'entropy']
    },
    'knn': {
        'n_neighbors': st.randint(1, 30),
        'weights': ['uniform', 'distance']
    },
    'adb': {
        'n_estimators': st.randint(50, 200),
        'learning_rate': st.uniform(0.01, 1.5)
    },
    'etc': {
        'n_estimators': st.randint(50, 200),
        'max_features': ['auto', 'sqrt', 'log2'],
        'max_depth': st.randint(4, 20),
        'criterion': ['gini', 'entropy']
    },
    'xgb': {
        'n_estimators': st.randint(50, 200),
        'learning_rate': st.uniform(0.01, 1.5),
        'max_depth': st.randint(3, 20)
    },
    'cat': {
        'depth': st.randint(4, 10),
        'learning_rate': st.uniform(0.01, 1.5),
        'iterations': st.randint(50, 200)
    },
    'dt': {
        'max_features': ['auto', 'sqrt', 'log2'],
        'max_depth': st.randint(4, 20),
        'criterion': ['gini', 'entropy']
    }
}

# Define base models
base_models = {
    'rf': RandomForestClassifier(random_state=random_state_value),
    'knn': KNeighborsClassifier(),
    'svc': SVC(probability=True, random_state=random_state_value),  # SVC without hyperparameter tuning
    'adb': AdaBoostClassifier(random_state=random_state_value),
    'gbc': GradientBoostingClassifier(random_state=random_state_value),  # GBC without hyperparameter tuning
    'etc': ExtraTreesClassifier(random_state=random_state_value),
    'xgb': XGBClassifier(random_state=random_state_value),
    'cat': CatBoostClassifier(verbose=0, random_state=random_state_value),
    'dt': DecisionTreeClassifier(random_state=random_state_value)
}

# Perform RandomizedSearchCV (excluding SVC and GBC) with timeout
best_estimators = []
for name, model in base_models.items():
    if name not in ['svc', 'gbc']:  # Skip hyperparameter tuning for SVC and GBC
        print(f"Hyperparameter tuning for {name}...")
        rand_search = RandomizedSearchCV(model, param_distributions=param_dist[name], n_iter=10, scoring='f1', cv=5, random_state=random_state_value)
        try:
            best_estimator = run_with_timeout(rand_search.fit, args=(X_train_scaled, y_train), timeout=max_search_time)
            best_estimators.append((name, best_estimator.best_estimator_))
        except TimeoutException:
            print(f"Timeout reached for {name}. Using default parameters.")
            best_estimators.append((name, model))
    else:
        best_estimators.append((name, model))

# Add the neural network wrapper
best_estimators.append(('nn', nn_wrapper))

# Create the stacking classifier
stacking_clf = StackingClassifier(
    estimators=best_estimators,
    final_estimator=LogisticRegression(),
    cv=5
)

# Train the StackingClassifier on the entire training data
print("Training StackingClassifier on the entire training data...")
stacking_clf.fit(X_train_scaled, y_train)

# Save the StackingClassifier model
print("Saving StackingClassifier model...")
stacking_model_path = "stacking_classifier_model.pkl"
joblib.dump(stacking_clf, stacking_model_path)

# Load the test data
print("Loading test data...")
test_file_path = 'test.csv'  # replace with your actual file path
test_data = pd.read_csv(test_file_path)
print("Test data loaded.")

# Handle missing values in the test data
X_test_final = test_data.fillna(test_data.mean())

# Normalize the test data
print("Normalizing test data...")
X_test_final_scaled = scaler.transform(X_test_final)
print("Test data normalized.")

# Make predictions with the StackingClassifier model
print("Making predictions with the StackingClassifier model...")
y_pred_final = stacking_clf.predict(X_test_final_scaled)

# Save predictions to a CSV file
print("Saving predictions...")
output = pd.DataFrame({'ID': test_data['ID'], 'Target': y_pred_final})
output.to_csv('predictions.csv', index=False)
print("Predictions saved to predictions.csv.")


Loading training data...
Training data loaded.
Normalizing features...
Features normalized.
Building neural network model...
Training neural network model...
Epoch 1/10000
46/67 [===================>..........] - ETA: 0s - loss: 0.7856 - f1_metric: 0.5438 
Epoch 1: val_loss improved from inf to 0.69440, saving model to best_nn_model.h5
67/67 [==============================] - 2s 6ms/step - loss: 0.7832 - f1_metric: 0.5442 - val_loss: 0.6944 - val_f1_metric: 0.4627 - lr: 0.0010
Epoch 2/10000
53/67 [======================>.......] - ETA: 0s - loss: 0.7507 - f1_metric: 0.5757
Epoch 2: val_loss did not improve from 0.69440


C:\Users\bapti\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


67/67 [==============================] - 0s 2ms/step - loss: 0.7410 - f1_metric: 0.5840 - val_loss: 0.6957 - val_f1_metric: 0.5102 - lr: 0.0010
Epoch 3/10000
45/67 [===================>..........] - ETA: 0s - loss: 0.7133 - f1_metric: 0.6174
Epoch 3: val_loss did not improve from 0.69440
67/67 [==============================] - 0s 3ms/step - loss: 0.7041 - f1_metric: 0.6343 - val_loss: 0.6947 - val_f1_metric: 0.6397 - lr: 0.0010
Epoch 4/10000
58/67 [========================>.....] - ETA: 0s - loss: 0.6880 - f1_metric: 0.6638
Epoch 4: val_loss improved from 0.69440 to 0.69294, saving model to best_nn_model.h5
67/67 [==============================] - 0s 3ms/step - loss: 0.6907 - f1_metric: 0.6645 - val_loss: 0.6929 - val_f1_metric: 0.6639 - lr: 0.0010
Epoch 5/10000
55/67 [=======================>......] - ETA: 0s - loss: 0.6971 - f1_metric: 0.6423
Epoch 5: val_loss improved from 0.69294 to 0.69036, saving model to best_nn_model.h5
67/67 [==============================] - 0s 3ms/step - lo

50/67 [=====================>........] - ETA: 0s - loss: 0.6441 - f1_metric: 0.7091
Epoch 28: val_loss improved from 0.66789 to 0.66705, saving model to best_nn_model.h5
67/67 [==============================] - 0s 4ms/step - loss: 0.6419 - f1_metric: 0.7079 - val_loss: 0.6671 - val_f1_metric: 0.7018 - lr: 0.0010
Epoch 29/10000
50/67 [=====================>........] - ETA: 0s - loss: 0.6457 - f1_metric: 0.7084
Epoch 29: val_loss improved from 0.66705 to 0.66480, saving model to best_nn_model.h5
67/67 [==============================] - 0s 3ms/step - loss: 0.6391 - f1_metric: 0.7139 - val_loss: 0.6648 - val_f1_metric: 0.7020 - lr: 0.0010
Epoch 30/10000
52/67 [======================>.......] - ETA: 0s - loss: 0.6367 - f1_metric: 0.7038
Epoch 30: val_loss did not improve from 0.66480
67/67 [==============================] - 0s 3ms/step - loss: 0.6386 - f1_metric: 0.7018 - val_loss: 0.6651 - val_f1_metric: 0.7070 - lr: 0.0010
Epoch 31/10000
46/67 [===================>..........] - ETA: 0s - 

48/67 [====================>.........] - ETA: 0s - loss: 0.5796 - f1_metric: 0.7463
Epoch 54: val_loss did not improve from 0.65313
67/67 [==============================] - 0s 3ms/step - loss: 0.5749 - f1_metric: 0.7467 - val_loss: 0.6555 - val_f1_metric: 0.7192 - lr: 1.0000e-04
Epoch 55/10000
53/67 [======================>.......] - ETA: 0s - loss: 0.5696 - f1_metric: 0.7523
Epoch 55: val_loss did not improve from 0.65313
67/67 [==============================] - 0s 2ms/step - loss: 0.5690 - f1_metric: 0.7580 - val_loss: 0.6556 - val_f1_metric: 0.7203 - lr: 1.0000e-04
Epoch 56/10000
55/67 [=======================>......] - ETA: 0s - loss: 0.5657 - f1_metric: 0.7538
Epoch 56: val_loss did not improve from 0.65313
67/67 [==============================] - 0s 3ms/step - loss: 0.5662 - f1_metric: 0.7480 - val_loss: 0.6550 - val_f1_metric: 0.7217 - lr: 1.0000e-04
Epoch 57/10000
49/67 [====================>.........] - ETA: 0s - loss: 0.5687 - f1_metric: 0.7395
Epoch 57: val_loss did not impr

...

Hyperparameter tuning for xgb...
Hyperparameter tuning for cat...


learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease l

Hyperparameter tuning for dt...


C:\Users\bapti\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
15 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\bapti\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\bapti\anaconda3\Lib\site-packages\sklearn\base.py", line 1144, in wrapper
    estimator._validate_params()
  File "C:\Users\bapti\anaconda3\Lib\site-packages\sklearn\base.py", line 637, in _validate_params
    validate_parameter_constraints(
  File "C:\Users\bapti\anaconda3\Lib\site-packages\

Training StackingClassifier on the entire training data...
17/17 [==============================] - 0s 917us/step
Saving StackingClassifier model...
Loading test data...
Test data loaded.
Normalizing test data...
Test data normalized.
Making predictions with the StackingClassifier model...
21/21 [==============================] - 0s 1ms/step
Saving predictions...
Predictions saved to predictions.csv.
